This is where I will process my data and run a model

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras import layers
from collections import defaultdict


import cv2 

from get_coords import Data

# do batch prediction function DONE
# mix features together DONE
# add color features DONE
# mean square error for regression DONE
# get started writing 
# get results 1/2 DONE

# Features to try: DONE
# 1) Raw Position Values DONE
# Find how they did visual complexity 
# Think about why some features are doing better
# try cross validation 
# reshuffle training and testing data
# try removing some percent of training and test on different testing sets and train on different training sets
# train should be different from test ALWAYS

# start writing related works 
# start writing methodology
# write indroduction
# write like other papers that I have read

# try to get positive results

# arxiv for posting negative result


Process Data

In [2]:
classes = ['button', 'field', 'heading', 'iframe', 'image', 'label', 'link', 'text']

directory = "dataset/images/english/"
#directory = "small_test/"

data_N = Data(
    directory=directory,
    pt=r'C:\Users\woprg\Desktop\CCIR\LearningML\TrainingYOLOv7_3\yolov5\runs\train\yolo_website6\weights\best.pt'
    )

raw_data = data_N.filter_confidence()
image_names = list(raw_data.keys())
image_path = data_N.images()
images = {}

for i in range(len(image_path)):
    images[image_names[i]] = cv2.imread(image_path[i])

def raw(d):
    rawData = {}
    temp = list(d.values())
    for i in range(len(temp)):
        temp_rawData = []
        for element in temp[i]:
            e = []
            e.append(float((float(element[0])+float(element[2]))/2))
            e.append(float((float(element[1])+float(element[3]))/2))
            e.append(float(element[5]))
            temp_rawData.extend(e)
        rawData[int(image_names[i])] = temp_rawData
    sizes = []
    for i in range(len(rawData)):
        sizes.append(len(rawData[i]))
    max_size = max(sizes)
    for i in range(len(rawData)):
        if len(rawData[i]) < max_size:
            for j in range(max_size - len(rawData[i])):
                rawData[i].append(0)
    return rawData
    
def raw():
    rawData = {}
    temp = list(raw_data.values())
    for i in range(len(temp)):
        temp_rawData = []
        for element in temp[i]:
            e = []
            e.append(float((float(element[0])+float(element[2]))/2))
            e.append(float((float(element[1])+float(element[3]))/2))
            e.append(float(element[5]))
            temp_rawData.extend(e)
        rawData[int(image_names[i])] = temp_rawData
    sizes = []
    for i in range(len(rawData)):
        sizes.append(len(rawData[i]))
    max_size = max(sizes)
    for i in range(len(rawData)):
        if len(rawData[i]) < max_size:
            for j in range(max_size - len(rawData[i])):
                rawData[i].append(0)
    return rawData

def colorfulness():
    colorfulness = {}
    for i in range(len(list(images.values()))):
        current_colorfulness = 0
        count = 0
        temp_image = list(images.values())[i] 
        hsv = cv2.cvtColor(temp_image, cv2.COLOR_BGR2HSV)
        for row in range(len(hsv)):
            for column in range(len(hsv[row])):
                current_colorfulness = current_colorfulness + hsv[row][column][1] * hsv[row][column][2]
                count = count + 1
        result = current_colorfulness/count
        colorfulness[int(list(images.keys())[i])] = result
    return colorfulness

def avgPos():
    temp = list(raw_data.values())
    avg_element_position = {}
    for i in range(len(temp)):
        avg_pixel_position_for_class = {}
        count = {}
        for element in temp[i]:
            if avg_pixel_position_for_class.get(int(element[5])) == None:
                avg_pixel_position_for_class[int(element[5])] = ((float(element[0])+float(element[2]))/2,(float(element[1])+float(element[3]))/2)
                count[int(element[5])] = 1
            else:
                count[int(element[5])] += 1
                avg_pixel_position_for_class[int(element[5])] = ((((float(element[0])+float(element[2]))/2)+avg_pixel_position_for_class[int(element[5])][0])/count[int(element[5])],(((float(element[1])+float(element[3]))/2)+avg_pixel_position_for_class[int(element[5])][0])/count[int(element[5])])
                
        avg_element_position[int(image_names[i])] = avg_pixel_position_for_class
        #print(count)

    return avg_element_position

def freqQuad():
    class_per_quadrant = {}
    temp = list(raw_data.values())
    for i in range(len(temp)):
        temp_data_holder = [{},{},{},{}]
        temp_data_holder_2 = []
        temp_image_name = image_names[i]
        temp_image = cv2.imread(directory + temp_image_name + '.png')
        w = temp_image.shape[0]
        h = temp_image.shape[1]
        for element in temp[i]:
            center_X = (element[0]+element[2])/2
            center_Y = (element[1]+element[3])/2
            if center_X >= w/2 and center_Y >= h/2:
                if temp_data_holder[0].get(int(element[5])) == None:
                    temp_data_holder[0][int(element[5])] = 1
                else:
                    temp_data_holder[0][int(element[5])] = temp_data_holder[0][int(element[5])] + 1
            if center_X < w/2 and center_Y >= h/2:
                if temp_data_holder[1].get(int(element[5])) == None:
                    temp_data_holder[1][int(element[5])] = 1
                else:
                    temp_data_holder[1][int(element[5])] = temp_data_holder[1][int(element[5])] + 1
            if center_X < w/2 and center_Y < h/2:
                if temp_data_holder[2].get(int(element[5])) == None:
                    temp_data_holder[2][int(element[5])] = 1
                else:
                    temp_data_holder[2][int(element[5])] = temp_data_holder[2][int(element[5])] + 1
            if center_X >= w/2 and center_Y < h/2:
                if temp_data_holder[3].get(int(element[5])) == None:
                    temp_data_holder[3][int(element[5])] = 1
                else:
                    temp_data_holder[3][int(element[5])] = temp_data_holder[3][int(element[5])] + 1
        for quadrant in temp_data_holder:
            for i in range(8):
                if quadrant.get(i) == None:
                    temp_data_holder_2.append(0)
                else:
                    temp_data_holder_2.append(quadrant[i])
        class_per_quadrant[int(temp_image_name)] = temp_data_holder_2
    return class_per_quadrant
    
def freqClass():
    dataX = {}
    temp = list(raw_data.values())
    for i in range(len(temp)):
        per_image_data = {}
        for element in temp[i]:
            if per_image_data.get(int(element[5])) == None:
                per_image_data[int(element[5])] = 1
            else:
                per_image_data[int(element[5])] = per_image_data[int(element[5])] + 1
        dataX[int(image_names[i])] = per_image_data
    return dataX


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\woprg/.cache\torch\hub\master.zip
YOLOv5  2022-11-2 Python-3.10.4 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
WARNING  NMS time limit 0.550s exceeded


Feature Mixer

In [3]:
# don't use raw
# do all combinations of average position, class frequency, and quadrants
# create table of results
# create narrative surrounding results
# base it off of related works and other papers

def mixFeatures(avgPosition,quadrants,classfreq,raw,modeltype, c):

    def createSVM(training, labels, apply_scaler2=True):
        try:
            for i in range(len(training)):
                training[i] = list(training[i].values())
        except Exception as e:
            training = training
        training_X = list(training)
        labels_X = list(labels)
        from sklearn import svm
        from sklearn.preprocessing import StandardScaler
        if apply_scaler2:
            scaler2 = StandardScaler()
            data2 = scaler2.fit_transform(training_X)
            #X = scaler2.fit(training)
            #data2 = scaler2.transform(training)
        else:
            data2 = training_X
        clf = svm.SVC()
        clf.fit(data2, labels_X)
        return clf

    def createLin(training, labels, normalize=True):
        try:
            for i in range(len(training)):
                training[i] = list(training[i].values())
        except Exception as e:
            training = training
        if normalize:
            normalizer = tf.keras.layers.Normalization(axis=-1)
            normalizer.adapt(tf.convert_to_tensor(training))
            linear_model = tf.keras.Sequential([
                normalizer,
                layers.Dense(units=1)
            ])
        else:
            linear_model = tf.keras.Sequential([
                layers.Dense(units=1)
            ])

        linear_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
            loss='mean_squared_logarithmic_error',
            metrics=[tf.keras.metrics.MeanSquaredError()])

        history = linear_model.fit(
            tf.convert_to_tensor(training),
            tf.convert_to_tensor(labels),
            epochs=100,
            validation_split=0.2,
            # Suppress logging.
            verbose=1)
        return linear_model

    def createRF(training, labels, estim=1000, r_state=42, apply_scaler3=True):
        # Import the model we are using
        try:
            for i in range(len(training)):
                training[i] = list(training[i].values())
        except Exception as e:
            training = training
        from sklearn.ensemble import RandomForestRegressor
        from sklearn.preprocessing import StandardScaler

        if apply_scaler3:
            scaler3 = StandardScaler()
            scaler3.fit(training)
            data3 = scaler3.transform(training)
        else:
            data3 = training
        # Instantiate model with 1000 decision trees
        rf = RandomForestRegressor(n_estimators = estim, random_state = r_state)
        # Train the model on training data
        rf.fit(data3, labels)
        return rf

    def formatAvgPos(SVM=False,COLOR=False):
        import numpy as np
        avg_pos_temp = avgPos()
        avg_pos = {}
        ratings = []
        if COLOR:
            c = colorfulness()

        for key, element in avg_pos_temp.items():
            if not data.get(key) == None:
                temp_data = []
                shape = cv2.imread(directory + str(key) + ".png").shape
                #print(shape)
                for i in range(8):
                    if element.get(i) == None:
                        temp_data.append(0)
                        temp_data.append(0)
                    else:
                        temp_data.append(element[i][0]
                        #/shape[1]
                        )
                        temp_data.append(element[i][1]
                        #/shape[0]
                        )
                if COLOR:
                    temp_data.append(c[int(key)])
                if SVM:
                    avg_pos[int(key)] = temp_data
                    if data[key] >= 4.5:
                        ratings.append(1)
                    else:
                        ratings.append(0)
                else:
                    avg_pos[int(key)] = temp_data
                    ratings.append((data[key]))
        return avg_pos,ratings 
        
    def formatFreq(SVM=False,RANDOMFOREST=False):
        import numpy as np
        class_freq_temp = freqClass()
        #print(class_freq_temp)
        data_2 = {}
        class_freq = {}

        for key, value in class_freq_temp.items():
            if not data.get(key) == None:
                temp_val = {0:[],1:[],2:[],3:[],4:[],5:[],6:[],7:[]}
                for i in range(8):
                    if value.get(i) == None:
                        temp_val[i] = 0
                    else:
                        temp_val[i] = value[i]
                class_freq[key] = temp_val

        for key in list(class_freq_temp.keys()):
            if not data.get(key) == None:
                data_2[key] = data[key]

        return class_freq, data_2

    def formatRaw(SVM=False, COLOR=False):
        import numpy as np

        freq_quad_temp = raw(raw_data)
        freq_quad = []
        quad_ratings = []
        quad_ratings_binary = []

        for key, value in freq_quad_temp.items():
            if not data.get(key) == None:
                freq_quad.append(value)
                quad_ratings.append(data[key])
                if data[key] >= 4:
                    quad_ratings_binary.append(1)
                else:
                    quad_ratings_binary.append(0)
        if SVM:
            return freq_quad_temp, quad_ratings_binary
        else:
            return freq_quad_temp, quad_ratings

    def formatQuad(SVM=False, COLOR=False):
        import numpy as np

        if COLOR:
            c = colorfulness()

        freq_quad_temp = freqQuad()
        freq_quad = {}
        quad_ratings = []
        quad_ratings_binary = []

        for key, value in freq_quad_temp.items():
            if not data.get(key) == None:
                if COLOR:
                    value.append(c[key])
                freq_quad[key] = value
                quad_ratings.append(data[key])
                if data[key] >= 4:
                    quad_ratings_binary.append(1)
                else:
                    quad_ratings_binary.append(0)
        if SVM:
            return freq_quad, quad_ratings_binary
        else:
            return freq_quad, quad_ratings

    def integrateData(input):
        for key, value in input.items():
            if training.get(key) == None:
                training[key] = value
            else:
                training[key].extend(value)
                
    def makeTrainTest(SVM=False, threshold=4):
        for key, value in training.items():
            if not data.get(key) == None:
                a.append(value)
                if SVM:
                    data_index = data[key]
                    if data_index > threshold:
                        b.append(1)
                    else:
                        b.append(0)
                else:
                    b.append(data[key])

    import csv
    path = r"dataset\preprocess\train_means_list.csv"
    data = {}
    with open(path) as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            if "english_resized" in row[0]:
                key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
                value = float(row[1])
                data[key] = value
    training = {}
    a = []
    b = []
    s = None
    r = None
    model = None
    if modeltype == "SVM":
        s = True
    else:
        s = False
    if modeltype == "RF":
        r = True
    else:
        r = False
    if avgPosition:
        integrateData(formatAvgPos(SVM=s,COLOR=c)[0])
    if quadrants:
        integrateData(formatQuad(SVM=s,COLOR=c)[0])
    if classfreq:
        integrateData(formatFreq(SVM=s,RANDOMFOREST=r)[0])
    if raw:
        integrateData(formatRaw(SVM=s,COLOR=c)[0])
    makeTrainTest(SVM=s)
    if r:
        model = createRF(a,b)
    if s:
        model = createSVM(a,b)
    else:
        model = createLin(a,b)
    return model

#m = mixFeatures(True,False,True,False,"SVM",False)

#m.predict([[0,1,0,2,3,1,3,5,1,3,7,3,1,8,2,6,3,0,0,0,0,0,0,0]])
print('Done feature mixer')



Done feature mixer


Mix Features Validation

In [4]:
from os import listdir
from os.path import isfile, join
from sklearn.metrics import mean_squared_error

def mixValid(x,avgPosition,quadrants,classfreq,raw,modeltype, c):
    def formatAvgPos_X(data_2=None, SVM=False):
        import numpy as np

        if data_2 == None:
            avg_pos_temp = avgPos()
        else:
            def avgPosOfGiven(d):
                temp = list(d.values())
                avg_element_position = {}
                for i in range(len(temp)):
                    avg_pixel_position_for_class = {}
                    count = {}
                    for element in temp[i]:
                        if avg_pixel_position_for_class.get(int(element[5])) == None:
                            avg_pixel_position_for_class[int(element[5])] = ((float(element[0])+float(element[2]))/2,(float(element[1])+float(element[3]))/2)
                            count[int(element[5])] = 1
                        else:
                            count[int(element[5])] += 1
                            avg_pixel_position_for_class[int(element[5])] = ((((float(element[0])+float(element[2]))/2)+avg_pixel_position_for_class[int(element[5])][0])/count[int(element[5])],(((float(element[1])+float(element[3]))/2)+avg_pixel_position_for_class[int(element[5])][0])/count[int(element[5])])
                            
                    avg_element_position[int(image_names[i])] = avg_pixel_position_for_class
                    #print(count)

                return avg_element_position
            avg_pos_temp = avgPosOfGiven(raw_data)
        avg_pos = {}
        ratings = []

        for key, element in avg_pos_temp.items():
            if not data.get(key) == None:
                temp_data = []
                shape = cv2.imread(directory + str(key) + ".png").shape
                #print(shape)
                for i in range(8):
                    if element.get(i) == None:
                        temp_data.append(0)
                        temp_data.append(0)
                    else:
                        temp_data.append(element[i][0]
                        #/shape[1]
                        )
                        temp_data.append(element[i][1]
                        #/shape[0]
                        )
                if SVM:
                    avg_pos[int(key)] = temp_data
                    if data[key] >= 4.5:
                        ratings.append(1)
                    else:
                        ratings.append(0)
                else:
                    avg_pos[int(key)] = temp_data
                    ratings.append((data[key]))
        return avg_pos,ratings
    def formatFreq_X(data_2=None, SVM=False,RANDOMFOREST=False):
        import numpy as np

        if data_2==None:
            class_freq_temp = freqClass()
        else:
            
            def freqClass(d):
                data = {}
                temp = list(d.values())
                for i in range(len(temp)):
                    per_image_data = {}
                    for element in temp[i]:
                        if per_image_data.get(int(element[5])) == None:
                            per_image_data[int(element[5])] = 1
                        else:
                            per_image_data[int(element[5])] = per_image_data[int(element[5])] + 1
                    data[int(image_names[i])] = per_image_data
                return data
            #class_freq_temp = freqClass(raw_data)
            class_freq_temp = freqClass(raw_data)
        #print(class_freq_temp)

        data_3 = {}

        class_freq = {}

        for key, value in class_freq_temp.items():
            if not data.get(key) == None:
                temp_val = {0:[],1:[],2:[],3:[],4:[],5:[],6:[],7:[]}
                for i in range(8):
                    if value.get(i) == None:
                        temp_val[i] = 0
                    else:
                        temp_val[i] = value[i]
                class_freq[key] = list(temp_val.values())

        for key in list(class_freq_temp.keys()):
            if not data.get(key) == None:
                data_3[key] = data[key]

        return class_freq, data_3
    def formatRaw_X(data_2=None, SVM=False):
        import numpy as np

        if data_2 == None:
            freq_quad_temp = raw(raw_data)
        else:
            freq_quad_temp=raw(data_2)
        freq_quad = []
        quad_ratings = []
        quad_ratings_binary = []

        for key, value in freq_quad_temp.items():
            if not data.get(key) == None:
                freq_quad.append(value)
                quad_ratings.append(data[key])
                if data[key] >= 4:
                    quad_ratings_binary.append(1)
                else:
                    quad_ratings_binary.append(0)
        if SVM:
            return freq_quad_temp, quad_ratings_binary
        else:
            return freq_quad_temp, quad_ratings
    def formatQuad_X(data_2=None, SVM=False):
        import numpy as np
        
        if data_2 == None:
            freq_quad_temp = freqQuad()
        else:
            def freqQuad(d):
                class_per_quadrant = {}
                temp = list(d.values())
                for i in range(len(temp)):
                    temp_data_holder = [{},{},{},{}]
                    temp_data_holder_2 = []
                    temp_image_name = image_names[i]
                    temp_image = cv2.imread(directory + temp_image_name + '.png')
                    w = temp_image.shape[0]
                    h = temp_image.shape[1]
                    for element in temp[i]:
                        center_X = (element[0]+element[2])/2
                        center_Y = (element[1]+element[3])/2
                        if center_X >= w/2 and center_Y >= h/2:
                            if temp_data_holder[0].get(int(element[5])) == None:
                                temp_data_holder[0][int(element[5])] = 1
                            else:
                                temp_data_holder[0][int(element[5])] = temp_data_holder[0][int(element[5])] + 1
                        if center_X < w/2 and center_Y >= h/2:
                            if temp_data_holder[1].get(int(element[5])) == None:
                                temp_data_holder[1][int(element[5])] = 1
                            else:
                                temp_data_holder[1][int(element[5])] = temp_data_holder[1][int(element[5])] + 1
                        if center_X < w/2 and center_Y < h/2:
                            if temp_data_holder[2].get(int(element[5])) == None:
                                temp_data_holder[2][int(element[5])] = 1
                            else:
                                temp_data_holder[2][int(element[5])] = temp_data_holder[2][int(element[5])] + 1
                        if center_X >= w/2 and center_Y < h/2:
                            if temp_data_holder[3].get(int(element[5])) == None:
                                temp_data_holder[3][int(element[5])] = 1
                            else:
                                temp_data_holder[3][int(element[5])] = temp_data_holder[3][int(element[5])] + 1
                    for quadrant in temp_data_holder:
                        for i in range(8):
                            if quadrant.get(i) == None:
                                temp_data_holder_2.append(0)
                            else:
                                temp_data_holder_2.append(quadrant[i])
                    class_per_quadrant[int(temp_image_name)] = temp_data_holder_2
                return class_per_quadrant
            freq_quad_temp = freqQuad(raw_data)
        freq_quad = {}
        quad_ratings = []
        quad_ratings_binary = []

        for key, value in freq_quad_temp.items():
            if not data.get(key) == None:
                freq_quad[key] = value
                quad_ratings.append(data[key])
                if data[key] >= 4:
                    quad_ratings_binary.append(1)
                else:
                    quad_ratings_binary.append(0)
        if SVM:
            return freq_quad, quad_ratings_binary
        else:
            return freq_quad, quad_ratings
    
    d = Data(directory=x,pt=r"C:\Users\woprg\Desktop\CCIR\LearningML\TrainingYOLOv7_3\yolov5\runs\train\yolo_website6\weights\best.pt")
    raw_data_X = d.filter_confidence()
    #print(raw_data)
    import csv
    path = r"dataset\preprocess\train_means_list.csv"
    data = {}
    with open(path) as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            if "english_resized" in row[0]:
                key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
                value = float(row[1])
                data[key] = value
    testing = {}

    #avgPosition,quadrants,classfreq,raw,modeltype,
    if avgPosition:
        aP = formatAvgPos_X(data_2=raw_data_X,SVM=False)[0]
        for key, value in aP.items():
            if testing.get(key) == None:
                testing[key] = value
            else:
                testing[key].extend(value)
    if quadrants:
        q = formatQuad_X(data_2=raw_data_X,SVM=False)[0]
        for key, value in q.items():
            if testing.get(key) == None:
                testing[key] = value
            else:
                testing[key].extend(value)
    if classfreq:
        f = formatFreq_X(data_2=raw_data_X,SVM=False)[0]
        for key, value in f.items():
            if testing.get(key) == None:
                testing[key] = value
            else:
                testing[key].extend(value)
    if raw:
        r = formatRaw_X(data_2=raw_data_X,SVM=False)[0]
        for key, value in r.items():
            if testing.get(key) == None:
                testing[key] = value
            else:
                testing[key].extend(value)
    model_type = modeltype
    model = mixFeatures(avgPosition,quadrants,classfreq,raw,model_type,False)
    predictions_temp = model.predict(list(testing.values()))
    predictions = []
    #print(predictions_temp)
    for prediction in predictions_temp:
        if model_type == "SVM":
            predictions.append(prediction)
        else:
            predictions.append(prediction[0])
    actual = []
    for key in list(testing.keys()):
        actual.append(data[key])
    def removeBadValues(act, pred):
        act_2 = act.copy()
        pred_2 = pred.copy()
        count_removed = 0
        for i in range(len(pred)):
            if pred[i]<0:
                pred_2.pop(i-count_removed)
                act_2.pop(i-count_removed)
                count_removed+=1
        return act_2, pred_2
    actual, predictions = removeBadValues(actual, predictions)

    if model_type == "SVM":
        for i in range(len(actual)):
            if actual[i] >= 4:
                actual[i] = 1
            else:
                actual[i] = 0

    accuracy = -1

    if model_type == "SVM":
        correct = 0
        for i in range(len(actual)):
            if actual[i] == predictions[i]:
                correct = correct + 1
        accuracy = float(correct/len(actual))
    else:
        accuracy = mean_squared_error(actual, predictions)

    return predictions, actual, accuracy, testing

# False, False, True, False is broken "TypeError: float() argument must be a string or a real number, not 'dict'"
# True, True, False, False and True, False, False, False are both 0.64
# False, True, False, False is 0.76

#mixValid("dataset/images/english/",True,True,True,False,"SVM",False)

In [16]:
with open("results.txt","w") as f:
    for model_aAabBbcCc in [
        #"LIN",
        "SVM",
        #"RF"
        ]:
        for aAa in [
            #True,
            False]:
            for bBb in [
                True,
                #False
                ]:
                for cCc in [
                    #True,
                    False
                    ]:
                    if(not aAa and not bBb and not cCc):
                        continue
                    variable_aaa = mixValid("dataset/images/english/",aAa,bBb,cCc,False,model_aAabBbcCc,False)
                    data_list = ""
                    for i in range(len(variable_aaa[0])):
                        '''
                        if variable_aaa[0][i] == 1:
                            data_list = str(variable_aaa[0][i]) + "," + str(list(variable_aaa[3].values())[i]).replace("[","").replace("]","").replace(" ","") + "\n" + data_list
                        else:
                            data_list = data_list + str(variable_aaa[0][i]) + "," + str(list(variable_aaa[3].values())[i]).replace("[","").replace("]","").replace(" ","") + "\n"
                        '''
                        data_list = data_list + str(variable_aaa[0][i]) + "," + str(list(variable_aaa[3].values())[i]).replace("[","").replace("]","").replace(" ","") + "\n"
                    f.write(model_aAabBbcCc + " " + str(aAa) + " " + str(bBb) + " " + str(cCc) + " " + str(variable_aaa[2]) + "\n"
                    #+ str(variable_aaa[3]) + "\n" + str(variable_aaa[0])
                    + data_list
                    )

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\woprg/.cache\torch\hub\master.zip
YOLOv5  2022-11-2 Python-3.10.4 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Data For McNemar's Test

In [ ]:
import csv
import random

with open("mcnemars.csv", "w", newline='') as csvfile:
    fieldnames = ['Instance', 'SVM Correct', 'Guess Correct']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for a1a in [True,False]:
        for b2b in [True,False]:
            for c3c in [True,False]:
                if(not a1a and not b2b and not c3c):
                    continue
                validation_data = mixValid("dataset/images/english/",a1a,b2b,c3c,False,"SVM",False)
                for i in range(len(validation_data[0])):
                    predicted = validation_data[0][i]
                    actual = validation_data[1][i]
                    guess = random.choice([0,1])
                    writer.writerow({"Instance":str(i),"SVM Correct":str(predicted==actual),"Guess Correct":str(guess==actual)})